In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
# Load the ann trained mode

model = load_model("model.h5",compile=True)

with open("one_hot_encode_geography.pkl",'rb') as file:
    lable_encoder_geo = pickle.load(file)

with open("label_encoder_gender.pkl",'rb') as file:
    label_encoder_gender = pickle.load(file)

In [4]:
df = pd.read_csv("Churn_Modelling.csv")
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [9]:
input_data = {
    'CreditScore':600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary': 50000
}

In [17]:
geo_encoded = lable_encoder_geo.transform([[input_data['Geography']]])
geo_encoded

e:\projects\Bank churn\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [19]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns = lable_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
input_df = pd.DataFrame([input_data])  
input_df = input_df.reset_index(drop=True)

input = pd.concat([input_df, geo_encoded_df], axis=1)

In [22]:
input

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [23]:
input['Gender'] = label_encoder_gender.transform(input['Gender'])
input

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [25]:
input.drop('Geography',axis = 1,inplace = True)

In [26]:
# Scale the data

with open("scaler.pkl",'rb') as file:
    scaler = pickle.load(file)

In [27]:
input_scaled = scaler.transform(input)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step


array([[0.03636804]], dtype=float32)